# Initialisation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder #, LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from FeatureNames import get_feature_names

# Exploration

In [2]:
! ls -lh ../02_data/

total 2,5G
-rw-rw-r-- 1 adrien adrien  26M juin  26  2018 application_test.csv
-rw-rw-r-- 1 adrien adrien 159M juin  26  2018 application_train.csv
-rw-rw-r-- 1 adrien adrien 359M juin  26  2018 bureau_balance.csv
-rw-rw-r-- 1 adrien adrien 163M juin  26  2018 bureau.csv
-rw-rw-r-- 1 adrien adrien 405M juin  26  2018 credit_card_balance.csv
-rw-rw-r-- 1 adrien adrien  37K juin  26  2018 HomeCredit_columns_description.csv
-rw-rw-r-- 1 adrien adrien 690M juin  26  2018 installments_payments.csv
-rw-rw-r-- 1 adrien adrien 375M juin  26  2018 POS_CASH_balance.csv
-rw-rw-r-- 1 adrien adrien 387M juin  26  2018 previous_application.csv
-rw-rw-r-- 1 adrien adrien 524K juin  26  2018 sample_submission.csv


In [3]:
col_desc = pd.read_csv('../02_data/HomeCredit_columns_description.csv', 
                       index_col=0)
col_desc

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...
217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


## Tables `application_{train|test}.csv`

Il y a plus de 200 colonnes pour 9 tables au format csv !
Avant d'aller plus loin dans l'exploration je vais me concentrer sur les tables principales : les tables `application_{train|test}.csv`.

Je vais d'abord regarder les plus grosses corrélations avec la variable `TARGET`

In [4]:
train = pd.read_csv('../02_data/application_train.csv')
test = pd.read_csv('../02_data/application_test.csv')
print('Dimensions jeu d\'entraînement :', train.shape)
print('Dimensions jeu de test : ', test.shape)
train.head()

Dimensions jeu d'entraînement : (307511, 122)
Dimensions jeu de test :  (48744, 121)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
assert len(train.SK_ID_CURR.unique()) == train.shape[0]
assert len(test.SK_ID_CURR.unique()) == test.shape[0]

train.set_index('SK_ID_CURR', inplace=True)
test.set_index('SK_ID_CURR', inplace=True)

test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print('name_col' + '\t' + 'data_type' + '\t' + 'dimensionality' + '\t' 
      + 'null_count' + '\t' + 'null_perct' + '\t'+ 'description')
for col in train.columns.tolist():
    column_typ = train[col].dtypes
    null_count = train[col].isna().sum()
    null_perct = null_count / train[col].isna().count()
    if train[col].dtype in ['object', 'int64']:
        dimensionality = train[col].nunique()
    else:
        dimensionality = np.nan
    desc = col_desc.loc[col_desc.Table.eq('application_{train|test}.csv')
                        & col_desc.Row.eq(col)].Description.tolist()[0]
    print(col + '\t'
          + str(column_typ) + '\t'
          + str(dimensionality) + '\t'
          + str(null_count) + '\t'
          + str(round(null_perct, 4) * 100) + '\t'
          + str(desc))

name_col	data_type	dimensionality	null_count	null_perct	description
TARGET	int64	2	0	0.0	Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)
NAME_CONTRACT_TYPE	object	2	0	0.0	Identification if loan is cash or revolving
CODE_GENDER	object	3	0	0.0	Gender of the client
FLAG_OWN_CAR	object	2	0	0.0	Flag if the client owns a car
FLAG_OWN_REALTY	object	2	0	0.0	Flag if client owns a house or flat
CNT_CHILDREN	int64	15	0	0.0	Number of children the client has
AMT_INCOME_TOTAL	float64	nan	0	0.0	Income of the client
AMT_CREDIT	float64	nan	0	0.0	Credit amount of the loan
AMT_ANNUITY	float64	nan	12	0.0	Loan annuity
AMT_GOODS_PRICE	float64	nan	278	0.09	For consumer loans it is the price of the goods for which the loan is given
NAME_TYPE_SUITE	object	7	1292	0.42	Who was accompanying client when he was applying for the loan
NAME_INCOME_TYPE	object	8	0	0.0	Clients incom

### Encodage des colonnes textuelles

In [7]:
print(train.dtypes.value_counts())

float64    65
int64      40
object     16
dtype: int64


In [8]:
categor_feats = train.select_dtypes('object').columns.tolist()
print(categor_feats)

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']


In [9]:
print(train[categor_feats].apply(pd.Series.nunique, 
                                 axis=0).sort_values(ascending=False))

ORGANIZATION_TYPE             58
OCCUPATION_TYPE               18
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
WEEKDAY_APPR_PROCESS_START     7
WALLSMATERIAL_MODE             7
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_EDUCATION_TYPE            5
FONDKAPREMONT_MODE             4
CODE_GENDER                    3
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
EMERGENCYSTATE_MODE            2
dtype: int64


#### Encodage des catégories multi-dimensionnelles

In [10]:
dimensionality = lambda x,df : df[[x]].apply(pd.Series.nunique).values

categor_feats_multidim = []
for feat in categor_feats:
    if dimensionality(feat,train) > 2:
        categor_feats_multidim.append(feat)
print(categor_feats_multidim)

['CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE']


In [11]:
for feat in categor_feats_multidim:
    if dimensionality(feat,train) <= 8:
        print(feat, train[feat].unique())

CODE_GENDER ['M' 'F' 'XNA']
NAME_TYPE_SUITE ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']
NAME_INCOME_TYPE ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
NAME_EDUCATION_TYPE ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS ['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']
NAME_HOUSING_TYPE ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
WEEKDAY_APPR_PROCESS_START ['WEDNESDAY' 'MONDAY' 'THURSDAY' 'SUNDAY' 'SATURDAY' 'FRIDAY' 'TUESDAY']
FONDKAPREMONT_MODE ['reg oper account' nan 'org spec account' 'reg oper spec account'
 'not specified']
HOUSETYPE_MODE ['block of flats' nan 'terraced house' 'specific housing']
WALLSMATERIAL_MODE ['Stone, brick' 'Block' nan 'Panel' 'Mixed' 'Wooden' 'Othe

On remarque deux choses :
1. La variable `CODE_GENDER` n'est pas vraiment multidimensionnelle
2. La variable `WEEKDAY_APPR_START` est catégorique ordinale, et devrait être traitée à part en tant que variable de temps
3. Dans les autres variables, il y a des espaces et des caractères spéciaux qu'il va falloir remplacer si on veut récupérer des dummy variables avec des noms simples à manipuler

On peut procéder comme ça :
* pour les espaces (` `) : remplacer par des `_`
* pour les `/` ou les `,` : remplacer par des `or`
* pour toutes les variables, inclure le nom des la variable au début de chaque valeur


In [12]:
categor_feats_multidim.remove('CODE_GENDER')
categor_feats_multidim.remove('WEEKDAY_APPR_PROCESS_START')
train_categor_multidim = train[categor_feats_multidim]
train_categor_multidim

,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE
SK_ID_CURR,,,,,,,,,,
100002,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,Business Entity Type 3,reg oper account,block of flats,"Stone, brick"
100003,Family,State servant,Higher education,Married,House / apartment,Core staff,School,reg oper account,block of flats,Block
100004,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,Government,NaN,NaN,NaN
100006,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,Business Entity Type 3,NaN,NaN,NaN
100007,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,Religion,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
456251,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,Services,reg oper account,block of flats,"Stone, brick"
456252,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,XNA,reg oper account,block of flats,"Stone, brick"
456253,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,School,reg oper account,block of flats,Panel


In [13]:
def format_categor_values(x):
    y = x.lower()
    y = y.replace(' ', '_')
    y = y.replace('-', '').replace(':', '')
    y = y.replace(',', '_or').replace('/', 'or')
    return y

print(train.NAME_HOUSING_TYPE.apply(format_categor_values).value_counts())

format_vfunc = np.vectorize(format_categor_values)
categor_value_formatter = FunctionTransformer(lambda x: format_vfunc(x))

#concat_feat_name_with_value = lambda x: '___' + x.name + '_' + x.astype(str)
print(categor_value_formatter.fit_transform(train.NAME_HOUSING_TYPE))

house_or_apartment     272868
with_parents            14840
municipal_apartment     11183
rented_apartment         4881
office_apartment         2617
coop_apartment           1122
Name: NAME_HOUSING_TYPE, dtype: int64
['house_or_apartment' 'house_or_apartment' 'house_or_apartment' ...
 'house_or_apartment' 'house_or_apartment' 'house_or_apartment']


In [14]:
categor_multidim_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('value_formatter', categor_value_formatter),
    ('encoder', OneHotEncoder())])
categor_multidim_preprocessor.fit_transform(train_categor_multidim)

<307511x127 sparse matrix of type '<class 'numpy.float64'>'
	with 3075110 stored elements in Compressed Sparse Row format>

In [15]:
feat_name_replacement = {k:v for k,v in zip(range(len(categor_feats_multidim)),
                                            categor_feats_multidim)}
onehot_feat_names = []
for feat_name in [n.replace('encoder__x', '')\
                  for n in get_feature_names(categor_multidim_preprocessor)]:
    for i in range(len(categor_feats_multidim)):
        if feat_name[0] == str(i):
            new_feat_name = feat_name_replacement[i] + feat_name[1:]
    onehot_feat_names.append(new_feat_name)

print(onehot_feat_names)

['NAME_TYPE_SUITE_children', 'NAME_TYPE_SUITE_family', 'NAME_TYPE_SUITE_group_of_people', 'NAME_TYPE_SUITE_other_a', 'NAME_TYPE_SUITE_other_b', 'NAME_TYPE_SUITE_spouse_or_partner', 'NAME_TYPE_SUITE_unaccompanied', 'NAME_TYPE_SUITE_unknown', 'NAME_INCOME_TYPE_businessman', 'NAME_INCOME_TYPE_commercial_associate', 'NAME_INCOME_TYPE_maternity_leave', 'NAME_INCOME_TYPE_pensioner', 'NAME_INCOME_TYPE_state_servant', 'NAME_INCOME_TYPE_student', 'NAME_INCOME_TYPE_unemployed', 'NAME_INCOME_TYPE_working', 'NAME_EDUCATION_TYPE_academic_degree', 'NAME_EDUCATION_TYPE_higher_education', 'NAME_EDUCATION_TYPE_incomplete_higher', 'NAME_EDUCATION_TYPE_lower_secondary', 'NAME_EDUCATION_TYPE_secondary_or_secondary_special', 'NAME_FAMILY_STATUS_civil_marriage', 'NAME_FAMILY_STATUS_married', 'NAME_FAMILY_STATUS_separated', 'NAME_FAMILY_STATUS_single_or_not_married', 'NAME_FAMILY_STATUS_unknown', 'NAME_FAMILY_STATUS_widow', 'NAME_HOUSING_TYPE_coop_apartment', 'NAME_HOUSING_TYPE_house_or_apartment', 'NAME_HOU

#### Encodage des catégories bi-dimensionnelles

In [16]:
categor_feats_binary = []
for feat in categor_feats:
    if dimensionality(feat,train) <= 2:
        categor_feats_binary.append(feat)
print(categor_feats_binary)

['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE']


In [17]:
categor_feats_binary.append('CODE_GENDER')
categor_feats_binary.append('WEEKDAY_APPR_PROCESS_START')
for feat in categor_feats_binary:
    print(feat, train[feat].unique())

NAME_CONTRACT_TYPE ['Cash loans' 'Revolving loans']
FLAG_OWN_CAR ['N' 'Y']
FLAG_OWN_REALTY ['Y' 'N']
EMERGENCYSTATE_MODE ['No' nan 'Yes']
CODE_GENDER ['M' 'F' 'XNA']
WEEKDAY_APPR_PROCESS_START ['WEDNESDAY' 'MONDAY' 'THURSDAY' 'SUNDAY' 'SATURDAY' 'FRIDAY' 'TUESDAY']


In [19]:
train_categor_binary = train[categor_feats_binary]

contract_types = ['Cash loans', 'Revolving loans']
y_or_n = ['N', 'Y']
yes_or_no = ['No', 'Yes']
genders = ['M', 'F']
weekdays = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY',
            'SUNDAY']
categories = [contract_types, y_or_n, y_or_n, yes_or_no, genders, weekdays]

categor_binary_preprocessor = Pipeline(steps=[
    ('nan_imputer', SimpleImputer(strategy='most_frequent')),
    ('xna_imputer', SimpleImputer(missing_values='XNA',
                                   strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=categories))])
categor_binary_preprocessor.fit_transform(train_categor_binary)

array([[0., 0., 1., 0., 0., 2.],
       [0., 0., 0., 0., 1., 0.],
       [1., 1., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 1., 3.],
       [0., 0., 1., 0., 1., 2.],
       [0., 0., 0., 0., 1., 3.]])

#### Pipeline finale des variables catégoriques

In [24]:
categor_preprocessor = make_column_transformer(
    (categor_binary_preprocessor, categor_feats_binary),
    (categor_multidim_preprocessor, categor_feats_multidim),
    remainder='passthrough'
)

categor_preprocessor.fit_transform(train)

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 0., 1.]])

In [25]:
train_encoded = categor_preprocessor.fit_transform(train)
print(train_encoded.shape)

(307511, 238)


In [22]:
binary_feat_names = get_feature_names(categor_binary_preprocessor)
print(binary_feat_names)

[]


In [26]:
train_encoded[:5]

array([[ 0.,  0.,  1., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ..., nan, nan, nan],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

### Pré-traitement des variables numériques

In [27]:
numeric_feats = train.select_dtypes(['int64', 'float64']).columns.tolist()
print(numeric_feats)

['TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',

In [12]:
desc = col_desc.loc[col_desc.Table.eq('application_{train|test}.csv')
                    & col_desc.Row.eq(col)].Description.tolist()

['What kind of occupation does the client have']


In [29]:
print(col_desc.loc[col_desc.Table.eq('application_{train|test}.csv')
                   & col_desc.Row.eq('CODE_GENDER')].Description.tolist())

['Gender of the client']


### Étude des variables

In [26]:
train.EXT_SOURCE_1.dtype

dtype('float64')

In [40]:
train.NAME_FAMILY_STATUS.value_counts()

Married                 196432
Single / not married     45444
Civil marriage           29775
Separated                19770
Widow                    16088
Unknown                      2
Name: NAME_FAMILY_STATUS, dtype: int64

In [41]:
train.NAME_EDUCATION_TYPE.value_counts()

Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: NAME_EDUCATION_TYPE, dtype: int64

In [54]:
train.NAME_TYPE_SUITE.value_counts()

Unaccompanied      248526
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other_A               866
Group of people       271
Name: NAME_TYPE_SUITE, dtype: int64

In [65]:
train['AGE'] = round(train['DAYS_BIRTH'] / - 365, 0).astype('int')

In [66]:
train.AGE

0         26
1         46
2         52
3         52
4         55
          ..
307506    26
307507    57
307508    41
307509    33
307510    46
Name: AGE, Length: 307511, dtype: int64

## Autres tables

In [22]:
col_desc.loc[col_desc.Table == 'bureau_balance.csv'].Description.values

array(['Recoded ID of Credit Bureau credit (unique coding for each application) - use this to join to CREDIT_BUREAU table ',
       'Month of balance relative to application date (-1 means the freshest balance date)',
       'Status of Credit Bureau loan during the month (active, closed, DPD0-30,� [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,� 5 means DPD 120+ or sold or written off ] )'],
      dtype=object)

In [12]:
bureau = pd.read_csv('../02_data/bureau.csv')
bureau_balance = pd.read_csv('../02_data/bureau_balance.csv')

In [13]:
bureau_balance.shape

(27299925, 3)

In [15]:
bureau_balance.shape[0] / 10 ** 3

27299.925

In [14]:
bureau_balance.columns

Index(['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'], dtype='object')

In [24]:
bureau.shape

(1716428, 17)

In [20]:
bureau = pd.read_csv('../02_data/bureau.csv')
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [22]:
bureau.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [32]:
col_desc.loc[col_desc.Row.eq('SK_ID_CURR') & col_desc.Table.eq('bureau.csv')].Description.values

array(['ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau '],
      dtype=object)

In [34]:
bureau.shape

(1716428, 17)

In [35]:
len(bureau.SK_ID_BUREAU.unique())

1716428

In [37]:
len(bureau.SK_ID_CURR.unique())

305811

In [40]:
bureau[bureau.duplicated(subset=['SK_ID_CURR']) == True]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.00,0.00,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.00,0.00,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.00,0.00,0.0,Consumer credit,-1508,NaN
